We scrap the reddit post with Python and Selenium. But it become incredibly hard to do for a number of reasons. 
1. Reddit doesn't rellt want everybody scraping their website even thougn it's public data. 
2. Evn if you were scraping it manually then you would have to parse all of that data. You'd have to understand the structured of it. 

In [ ]:
import setup

setup.init()

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from snapshots.models import BrightDataSnapshot

In [ ]:
BRIGHT_DATA_REDDIT_SCRAPER_API_KEY = os.environ.get("BRIGHT_DATA_REDDIT_SCRAPER_API_KEY")
assert BRIGHT_DATA_REDDIT_SCRAPER_API_KEY is not None

In [ ]:
def get_crawl_headers():
    return {
	"Authorization": f"Bearer {BRIGHT_DATA_REDDIT_SCRAPER_API_KEY}",
	"Content-Type": "application/json",
}

In [ ]:
import requests


def perform_scrape_snapshot(subreddit_url, num_of_posts: int = 20):
    url = "https://api.brightdata.com/datasets/v3/trigger"
    dataset_id = "gd_lvz8ah06191smkebj4"
    headers = get_crawl_headers()
    params = {
    	"dataset_id": dataset_id,
    	"include_errors": "true",
    	"type": "discover_new",
    	"discover_by": "subreddit_url",
    	"limit_per_input": "100",
    }
    data = [
    	{"url": f"{subreddit_url}","sort_by":"Top","sort_by_time":"Today","num_of_posts":num_of_posts},
    ]
    
    response = requests.post(url, headers=headers, params=params, json=data)
    response.raise_for_status()
    data = response.json()
    snapshot_id = data.get('snapshot_id')
    BrightDataSnapshot.objects.create(
        snapshot_id = snapshot_id,
        dataset_id = dataset_id,
        status="Unknown"
    )
    return snapshot_id


perform_scrape_snapshot("https://www.reddit.com/r/python", num_of_posts=25)
    

"sd_mh26rkjnjg0u6fkq3"

In [ ]:
snapshot_id = "sd_mh34c3v6hbr1havmj"

In [ ]:
def get_snapshot_progress(snapshot_id: str) -> bool:
    url = f"https://api.brightdata.com/datasets/v3/progress/{snapshot_id}"
    headers = get_crawl_headers()
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    snapshot_id = data.get('snapshot_id')
    dataset_id= data.get('dataset_id')
    BrightDataSnapshot.objects.update_or_create(
        snapshot_id=snapshot_id,
        dataset_id=dataset_id,
        defaults = {
            "status": data.get('status')
        }
    )
    return data.get('status') == 'ready'


get_snapshot_progress(snapshot_id)

In [ ]:
def download_snapshot(snapshot_id: str) -> dict:
    url = f"https://api.brightdata.com/datasets/v3/snapshot/{snapshot_id}"
    headers = get_crawl_headers()
    params = {
        "format": "json"
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    return data

reddit_results = download_snapshot(snapshot_id)

In [ ]:
# reddit_results

In [ ]:
model_field_names = [field.name for field in RedditPost._meta.get_fields()]
model_field_names

In [ ]:
skip_fields = ['id', 'post_id',
 'url',
 'description',
 'comments',
 'related_posts',
 'community_name',
 'num_upvotes',
 'num_comments']

valid_fields = [x for x in model_field_names if x not in skip_fields]
valid_fields

In [ ]:
for thread in reddit_results:
    post_id = thread.get("post_id")
    url = thread.get("url")
    update_data = {k:v for k, v in thread.items() if k in valid_fields}
    print(post_id, url, update_data)
    RedditPost.objects.update_or_create(
        post_id=post_id,
        url=url,
        defaults=update_data
    )